In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.plotly
import plotly.graph_objs as go
import scipy.stats as stats
# Work with Plotly Offline
from plotly.offline import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_directed = pd.read_csv("Centrality_Directed_LinkNoWeight_07052019.csv", sep=";")
df_directed = df_directed[df_directed['Layer'] == "1-Multi"]
df_directed.drop(labels=["Layer", "Node"], axis=1, inplace=True)
# Make list of column name
l_col = list(df_directed.columns[1:])

In [3]:
df_directed.head()

,Label,Strength,StrengthIn,StrengthOut,Degree,DegreeIn,DegreeOut,PageRank,Eigenvector,Hub,Authority,Katz,Multiplexity,Kcore
0,Aalborg,18,9,9,18,9,9,0.047119,0.047750,0.046597,0.049215,0.048421,0.333333,6
1,Aarau,24,12,12,24,12,12,0.078256,0.001444,0.001425,0.001443,0.001458,0.333333,6
2,Aarhus,74,37,37,74,37,37,0.114862,0.218722,0.209097,0.221463,0.213328,0.333333,25
3,Aberdeen,33,15,18,33,15,18,0.053496,0.050846,0.094582,0.052966,0.099118,0.666667,18
4,Abidjan,36,18,18,36,18,18,0.061291,0.160512,0.154374,0.157745,0.154545,0.333333,22


In [4]:
'''
df_directed['Multiplexity'], df_directed['Kcore'] = df_directed['Kcore'], df_directed['Multiplexity']
df_directed.rename(columns={'Multiplexity':'Kcore', 'Kcore':'Multiplexity'}, inplace=True)
df_directed.head()
''';

In [5]:
df_directed['Strength'], df_directed['StrengthIn'], df_directed['PageRank'], df_directed['Eigenvector'], \
df_directed['Degree'], df_directed['StrengthOut'] = \
df_directed['PageRank'], df_directed['Eigenvector'], df_directed['Strength'], df_directed['StrengthIn'], \
df_directed['StrengthOut'], df_directed['Degree']

df_directed.rename(columns={'Strength':'PageRank', 'StrengthIn':'Eigenvector', 'PageRank':'Strength', \
                           'Eigenvector':'StrengthIn', 'Degree':'StrengthOut', 'StrengthOut':'Degree'}, inplace=True)
df_directed.head()

,Label,PageRank,Eigenvector,Degree,StrengthOut,DegreeIn,DegreeOut,Strength,StrengthIn,Hub,Authority,Katz,Multiplexity,Kcore
0,Aalborg,0.047119,0.047750,18,9,9,9,18,9,0.046597,0.049215,0.048421,0.333333,6
1,Aarau,0.078256,0.001444,24,12,12,12,24,12,0.001425,0.001443,0.001458,0.333333,6
2,Aarhus,0.114862,0.218722,74,37,37,37,74,37,0.209097,0.221463,0.213328,0.333333,25
3,Aberdeen,0.053496,0.050846,33,18,15,18,33,15,0.094582,0.052966,0.099118,0.666667,18
4,Abidjan,0.061291,0.160512,36,18,18,18,36,18,0.154374,0.157745,0.154545,0.333333,22


In [6]:
'''
df_directed['Strength'], df_directed['DegreeIn'], df_directed['DegreeOut'], df_directed['StrengthOut'] = \
df_directed['StrengthOut'], df_directed['DegreeOut'], df_directed['DegreeIn'], df_directed['Strength']

df_directed.rename(columns={'Strength':'StrengthOut', 'DegreeIn':'DegreeOut', 'DegreeOut':'DegreeIn', \
                           'SrengthOut':'Strength'}, inplace=True)

df_directed.head()
''';

In [7]:
# Delete StrengthIn, StrengthOut, DegreeIn, DegreeOut
# It is basically just telling the redundant information of total strength and degree
#df_directed.drop(labels=['StrengthIn', 'StrengthOut', 'DegreeIn', 'DegreeOut'], axis = 1, inplace=True)
#df_directed.head()

In [8]:
df_directed['Multiplexity'] = df_directed['Multiplexity'].round(2)

In [9]:
# split data table into data X and class labels y

X = df_directed.iloc[:,1:13].values
y = df_directed.iloc[:,13].values

In [10]:
y_columns = list(df_directed.iloc[:,1:13].columns)

In [11]:
df_directed.head()

,Label,PageRank,Eigenvector,Degree,StrengthOut,DegreeIn,DegreeOut,Strength,StrengthIn,Hub,Authority,Katz,Multiplexity,Kcore
0,Aalborg,0.047119,0.047750,18,9,9,9,18,9,0.046597,0.049215,0.048421,0.33,6
1,Aarau,0.078256,0.001444,24,12,12,12,24,12,0.001425,0.001443,0.001458,0.33,6
2,Aarhus,0.114862,0.218722,74,37,37,37,74,37,0.209097,0.221463,0.213328,0.33,25
3,Aberdeen,0.053496,0.050846,33,18,15,18,33,15,0.094582,0.052966,0.099118,0.67,18
4,Abidjan,0.061291,0.160512,36,18,18,18,36,18,0.154374,0.157745,0.154545,0.33,22


# Exploratory Visualization

In [12]:
# To Do Later
# https://plot.ly/ipython-notebooks/principal-component-analysis/

# Standarizing

In [13]:
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)

# 1. Eigendecomposition -  Computing Eigenvectors and Eigenvalues

In [14]:
# Covariance Matrix

In [15]:
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
print('Covariance matrix \n%s' %cov_mat)

Covariance matrix 
[[1.0017452  0.64660253 0.94217163 0.9415676  0.9421654  0.9415676
  0.94217163 0.9421654  0.65578398 0.64841917 0.65194312 0.43381238]
 [0.64660253 1.0017452  0.82634306 0.82263654 0.8295592  0.82263654
  0.82634306 0.8295592  0.9983651  1.0016575  0.99789076 0.28727514]
 [0.94217163 0.82634306 1.0017452  1.0014231  1.00141393 1.0014231
  1.0017452  1.00141393 0.83632343 0.82856334 0.83443491 0.45380988]
 [0.9415676  0.82263654 1.0014231  1.0017452  1.00043873 1.0017452
  1.0014231  1.00043873 0.83417141 0.82493431 0.83230228 0.45512356]
 [0.9421654  0.8295592  1.00141393 1.00043873 1.0017452  1.00043873
  1.00141393 1.0017452  0.83795656 0.83169945 0.83604963 0.45217959]
 [0.9415676  0.82263654 1.0014231  1.0017452  1.00043873 1.0017452
  1.0014231  1.00043873 0.83417141 0.82493431 0.83230228 0.45512356]
 [0.94217163 0.82634306 1.0017452  1.0014231  1.00141393 1.0014231
  1.0017452  1.00141393 0.83632343 0.82856334 0.83443491 0.45380988]
 [0.9421654  0.8295592  1.0

In [16]:
# Pake function numpy untuk covariance
print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))

NumPy covariance matrix: 
[[1.0017452  0.64660253 0.94217163 0.9415676  0.9421654  0.9415676
  0.94217163 0.9421654  0.65578398 0.64841917 0.65194312 0.43381238]
 [0.64660253 1.0017452  0.82634306 0.82263654 0.8295592  0.82263654
  0.82634306 0.8295592  0.9983651  1.0016575  0.99789076 0.28727514]
 [0.94217163 0.82634306 1.0017452  1.0014231  1.00141393 1.0014231
  1.0017452  1.00141393 0.83632343 0.82856334 0.83443491 0.45380988]
 [0.9415676  0.82263654 1.0014231  1.0017452  1.00043873 1.0017452
  1.0014231  1.00043873 0.83417141 0.82493431 0.83230228 0.45512356]
 [0.9421654  0.8295592  1.00141393 1.00043873 1.0017452  1.00043873
  1.00141393 1.0017452  0.83795656 0.83169945 0.83604963 0.45217959]
 [0.9415676  0.82263654 1.0014231  1.0017452  1.00043873 1.0017452
  1.0014231  1.00043873 0.83417141 0.82493431 0.83230228 0.45512356]
 [0.94217163 0.82634306 1.0017452  1.0014231  1.00141393 1.0014231
  1.0017452  1.00141393 0.83632343 0.82856334 0.83443491 0.45380988]
 [0.9421654  0.82955

In [17]:
# Eigendecomposition of covariance matrix
cov_mat = np.cov(X_std.T)

eig_vals, eig_vecs = np.linalg.eig(cov_mat)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

Eigenvectors 
[[-2.79147973e-01+0.00000000e+00j -3.20479557e-01+0.00000000e+00j
   3.02537971e-01+0.00000000e+00j -8.47493611e-01+0.00000000e+00j
   9.53378610e-02+0.00000000e+00j  1.07808958e-02+0.00000000e+00j
  -1.93407226e-02+0.00000000e+00j  3.87540947e-03+0.00000000e+00j
   1.97417328e-13+0.00000000e+00j  3.68470480e-14+1.07344351e-17j
   3.68470480e-14-1.07344351e-17j  1.27007316e-14+0.00000000e+00j]
 [-2.85645284e-01+0.00000000e+00j  3.61622371e-01+0.00000000e+00j
  -1.78337642e-01+0.00000000e+00j -1.59805328e-01+0.00000000e+00j
  -4.39924446e-01+0.00000000e+00j  2.56145368e-01+0.00000000e+00j
   4.25619820e-01+0.00000000e+00j  5.38514416e-01+0.00000000e+00j
  -3.42922900e-12+0.00000000e+00j -1.37041295e-12-1.28042515e-14j
  -1.37041295e-12+1.28042515e-14j -4.76765164e-13+0.00000000e+00j]
 [-3.09085997e-01+0.00000000e+00j -1.29346776e-01+0.00000000e+00j
   1.28040396e-01+0.00000000e+00j  1.93615236e-01+0.00000000e+00j
  -2.41507072e-02+0.00000000e+00j -6.84641007e-04+0.00000000

In [18]:
# Correlation matrix
cor_mat1 = np.corrcoef(X_std.T)

eig_vals, eig_vecs = np.linalg.eig(cor_mat1)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

Eigenvectors 
[[-2.79147973e-01+0.00000000e+00j -3.20479557e-01+0.00000000e+00j
   3.02537971e-01+0.00000000e+00j -8.47493611e-01+0.00000000e+00j
   9.53378610e-02+0.00000000e+00j  1.07808958e-02+0.00000000e+00j
  -1.93407226e-02+0.00000000e+00j  3.87540947e-03+0.00000000e+00j
  -2.00658334e-13+0.00000000e+00j  7.47148997e-15-2.96550182e-16j
   7.47148997e-15+2.96550182e-16j -1.16877137e-15+0.00000000e+00j]
 [-2.85645284e-01+0.00000000e+00j  3.61622371e-01+0.00000000e+00j
  -1.78337642e-01+0.00000000e+00j -1.59805328e-01+0.00000000e+00j
  -4.39924446e-01+0.00000000e+00j  2.56145368e-01+0.00000000e+00j
   4.25619820e-01+0.00000000e+00j  5.38514416e-01+0.00000000e+00j
   4.99957480e-12+0.00000000e+00j  2.14896205e-13-3.25613120e-15j
   2.14896205e-13+3.25613120e-15j -2.45479705e-14+0.00000000e+00j]
 [-3.09085997e-01+0.00000000e+00j -1.29346776e-01+0.00000000e+00j
   1.28040396e-01+0.00000000e+00j  1.93615236e-01+0.00000000e+00j
  -2.41507072e-02+0.00000000e+00j -6.84641007e-04+0.00000000

In [19]:
cor_mat2 = np.corrcoef(X.T)

eig_vals, eig_vecs = np.linalg.eig(cor_mat2)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

Eigenvectors 
[[-2.79147973e-01 -3.20479557e-01  3.02537971e-01 -8.47493611e-01
   9.53378610e-02  1.07808958e-02 -1.93407226e-02 -3.87540947e-03
  -2.18782955e-14  2.86106210e-14  1.55314736e-14 -8.88256927e-15]
 [-2.85645284e-01  3.61622371e-01 -1.78337642e-01 -1.59805328e-01
  -4.39924446e-01  2.56145368e-01  4.25619820e-01 -5.38514416e-01
  -5.20289802e-12  8.65048375e-12  3.71152697e-12 -2.12135544e-12]
 [-3.09085997e-01 -1.29346776e-01  1.28040396e-01  1.93615236e-01
  -2.41507072e-02 -6.84641005e-04  3.85262386e-03 -3.19447731e-04
  -8.77859877e-01  5.91367133e-01  6.46742486e-01 -3.67002773e-01]
 [-3.08700359e-01 -1.33214328e-01  1.28862745e-01  2.27448638e-01
   2.01566965e-01  4.36552858e-01 -4.52590092e-02 -2.86704236e-02
   3.45674698e-01 -3.01972547e-01  3.79780240e-01 -1.01031160e-01]
 [-3.09274080e-01 -1.25339646e-01  1.27122338e-01  1.59176815e-01
  -2.53040520e-01 -4.44097166e-01  5.36553718e-02  2.84321628e-02
   1.31036631e-01 -2.85996728e-01 -1.59386492e-01  6.89693

In [20]:
# Singular vector decomposition for computation efficiency
u,s,v = np.linalg.svd(X_std.T)
u

array([[-2.79147973e-01,  3.20479557e-01,  3.02537971e-01,
         8.47493611e-01,  9.53378610e-02, -1.07808958e-02,
        -1.93407226e-02,  3.87540947e-03,  2.62218278e-16,
         1.46912473e-16,  1.49839016e-16,  5.61896310e-17],
       [-2.85645284e-01, -3.61622371e-01, -1.78337642e-01,
         1.59805328e-01, -4.39924446e-01, -2.56145368e-01,
         4.25619820e-01,  5.38514416e-01, -4.99596668e-16,
         1.88753116e-16, -9.92533446e-17,  7.12246085e-17],
       [-3.09085997e-01,  1.29346776e-01,  1.28040396e-01,
        -1.93615236e-01, -2.41507072e-02,  6.84641006e-04,
         3.85262386e-03,  3.19447742e-04, -5.77668171e-01,
        -7.06769503e-01,  2.25234636e-03, -5.16390804e-03],
       [-3.08700359e-01,  1.33214328e-01,  1.28862745e-01,
        -2.27448638e-01,  2.01566965e-01, -4.36552858e-01,
        -4.52590092e-02,  2.86704236e-02,  2.89747090e-01,
         1.15446338e-03,  6.70436839e-01,  2.26055891e-01],
       [-3.09274080e-01,  1.25339646e-01,  1.2712233

# 2. Selecting Principal Components

In [21]:
#for ev in eig_vecs:
#    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev), decimal=2)
#print('Everything ok!')

In [22]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
#eig_pairs.sort()
#eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

Eigenvalues in descending order:
10.135188914033767
1.1388718301789094
0.6732298974611617
0.04395364493018815
0.00765080324114912
0.0009607061104534419
0.00012423105290712682
1.9972991460514424e-05
1.3216322615207365e-18
4.69584107369373e-17
1.1120368250936917e-18
6.330599210808456e-20


In [23]:
# Check top 5 K-Core value based on cluster members
top_kcore = df_directed.groupby(by='Kcore').count()['Label'].sort_values(ascending=False).head(5)

In [24]:
top_kcore

Kcore
2     116
4      82
6      63
8      34
36     34
Name: Label, dtype: int64

In [25]:
# check the top K-Core value
list(set(df_directed['Kcore']))[-5:]

[32, 33, 34, 36, 38]

In [26]:
top_kcore_index = tuple(list(top_kcore.index))

In [27]:
'''
colors_top_kcore = {2: '#0D76BF', 
          4: '#1b4ee7', 
          6: '#39da2b',
          8: '#050505',
          36: '#c90dfd',
          12: '#c90dfd',
          10: '#c90dfd',
          18: '#c90dfd',
          16: '#c90dfd',
          22: '#c90dfd'}
'''

colors_top_kcore = {38: '#fe0000', 
          36: '#03dac5', 
          34: '#0bff01',
          33: '#011efe',
          32: '#fe00f6'}

In [28]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = dict(
    type='bar',
    x=['PC %s' %i for i in range(1,13)],
    #x=['PC-%s' %i for i in y_columns],
    y=var_exp,
    name='Individual'
)

trace2 = dict(
    type='scatter',
    x=['PC %s' %i for i in range(1,13)], 
    #x=['PC-%s' %i for i in y_columns],
    y=cum_var_exp,
    name='Cumulative'
)

data = [trace1, trace2]

layout=dict(
    title='Explained variance by different principal components',
    yaxis=dict(
        title='Explained variance in percent'
    ),
    annotations=list([
        dict(
            x=1.16,
            y=1.05,
            xref='paper',
            yref='paper',
            text='Explained Variance',
            showarrow=False,
        )
    ])
)

fig = dict(data=data, layout=layout)
plotly.offline.plot(fig, filename='selecting-principal-components')

/Users/andreas.yunus/anaconda/envs/py36/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `selecting-principal-components` didn't end with .html. Adding .html to the end of your file.



'file:///Users/andreas.yunus/Desktop/Graduation/selecting-principal-components.html'

In [29]:
'''
matrix_w = np.hstack((eig_pairs[0][1].reshape(8,1), 
                      eig_pairs[1][1].reshape(8,1)))

print('Matrix W:\n', matrix_w)
''';

In [30]:
# Projection into a new space
# Unused - replaced by Scikit learn algorithm
'''
Y = X_std.dot(matrix_w)

data = []

for name, col in zip((0.33, 0.67, 1.00), colors.values()):
    trace = dict(
        type='scatter',
        x=Y[y==name,0],
        y=Y[y==name,1],
        mode='markers',
        name=name,
        marker=dict(
            color=col,
            size=12,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5),
            opacity=0.8)
    )
    data.append(trace)

layout = dict(
    showlegend=True,
    scene=dict(
        xaxis=dict(title='PC1'),
        yaxis=dict(title='PC2')
    )
)

fig = dict(data=data, layout=layout)
plotly.offline.plot(fig, filename='projection-matrix')
''';

# Top K-Core

In [31]:
# SK Learn Implementation

In [32]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=2)
Y_sklearn = sklearn_pca.fit_transform(X_std)

In [33]:
X_std

array([[-0.49367913, -0.24313016, -0.42726796, ..., -0.23391572,
        -0.2321601 , -0.45066581],
       [ 0.04077885, -0.56332412, -0.27155252, ..., -0.56542232,
        -0.56251055, -0.45066581],
       [ 0.66913554,  0.93907575,  1.02607609, ...,  0.96138727,
         0.92782862, -0.45066581],
       ...,
       [ 2.56299932,  0.87567751,  2.5053727 , ...,  0.90388243,
         0.99503987,  3.99450111],
       [-0.2491127 , -0.57265737, -0.53107825, ..., -0.57477223,
        -0.57203247, -0.45066581],
       [-0.2985703 , -0.56784207, -0.37536281, ..., -0.56995759,
        -0.56713402, -0.45066581]])

In [34]:
data = []

for name, col in zip((38,36,34,33,32), colors_top_kcore.values()):

    trace = dict(
        type='scatter',
        x=Y_sklearn[y==name,0],
        y=Y_sklearn[y==name,1],
        mode='markers',
        name=name,
        marker=dict(
            color=col,
            size=12,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5),
            opacity=0.8)
    )
    data.append(trace)

layout = dict(
        xaxis=dict(title='PC1', showline=False),
        yaxis=dict(title='PC2', showline=False)
)
fig = dict(data=data, layout=layout)
plotly.offline.plot(fig, filename='pca-scikitlearn')

/Users/andreas.yunus/anaconda/envs/py36/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `pca-scikitlearn` didn't end with .html. Adding .html to the end of your file.



'file:///Users/andreas.yunus/Desktop/Graduation/pca-scikitlearn.html'

In [35]:
df_directed.head()

,Label,PageRank,Eigenvector,Degree,StrengthOut,DegreeIn,DegreeOut,Strength,StrengthIn,Hub,Authority,Katz,Multiplexity,Kcore
0,Aalborg,0.047119,0.047750,18,9,9,9,18,9,0.046597,0.049215,0.048421,0.33,6
1,Aarau,0.078256,0.001444,24,12,12,12,24,12,0.001425,0.001443,0.001458,0.33,6
2,Aarhus,0.114862,0.218722,74,37,37,37,74,37,0.209097,0.221463,0.213328,0.33,25
3,Aberdeen,0.053496,0.050846,33,18,15,18,33,15,0.094582,0.052966,0.099118,0.67,18
4,Abidjan,0.061291,0.160512,36,18,18,18,36,18,0.154374,0.157745,0.154545,0.33,22


In [36]:
# should I see the PCA and investigate the clusters by degree of K-core (yg paling besar)
# atau by cluster in K-core - by number of members

# Adding TEU data to focus on 15 European Hubs

In [37]:
# Rank the ports based on Borda Count Methods
# Preference-based Ranking
# List of Busiest Port in Europe as per TEU in 2018. No data for 2019
# https://www.porteconomics.eu/2019/03/02/portgraphic-top15-container-ports-in-europe-in-2018/

# Make dictionary
port_teu_dict = {"Ports":["Rotterdam", "Antwerp", "Hamburg", 
            "Bremerhaven", "Valencia", "Piraeus",
           "Algeciras", "Felixstowe", "Barcelona",
           "Marsaxlokk", "Le_Havre", "Genoa",
           "Gioia_Tauro", "Southampton","Gdansk"], "TEU":[14513,11100,8730,5467,5104,4908,4772,4161,3423,3310,2884,2609,2301,1995,1949]}

port_teu = pd.DataFrame.from_dict(data=port_teu_dict)

df_directed.set_index('Label', inplace=True)

df_top15 = df_directed.loc[list(port_teu_dict['Ports'])]

# Remove Genoa from list - data not available - NaN
df_top15.dropna(inplace=True)

# Remove genoa TEU
genoa = list(port_teu['TEU'])
genoa.pop(11)

# add TEU data
df_top15['TEU'] = genoa

/Users/andreas.yunus/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike



In [38]:
df_top15.head()

,PageRank,Eigenvector,Degree,StrengthOut,DegreeIn,DegreeOut,Strength,StrengthIn,Hub,Authority,Katz,Multiplexity,Kcore,TEU
Label,,,,,,,,,,,,,,
Rotterdam,1.000000,1.000000,553.0,279.0,274.0,279.0,553.0,274.0,1.000000,1.000000,1.000000,1.00,36.0,14513
Antwerp,0.683192,0.842618,384.0,195.0,189.0,195.0,384.0,189.0,0.843401,0.844225,0.841935,1.00,36.0,11100
Hamburg,0.508203,0.717935,297.0,148.0,149.0,148.0,297.0,149.0,0.695131,0.718912,0.707992,1.00,36.0,8730
Bremerhaven,0.382071,0.586279,243.0,125.0,118.0,125.0,243.0,118.0,0.627794,0.590359,0.622340,0.67,36.0,5467
Valencia,0.187045,0.729839,140.0,69.0,71.0,69.0,140.0,71.0,0.692398,0.722064,0.695040,0.67,36.0,5104


In [39]:
df_top15.columns.to_list()

['PageRank',
 'Eigenvector',
 'Degree',
 'StrengthOut',
 'DegreeIn',
 'DegreeOut',
 'Strength',
 'StrengthIn',
 'Hub',
 'Authority',
 'Katz',
 'Multiplexity',
 'Kcore',
 'TEU']

In [40]:
df_top15.drop(labels=['StrengthIn',
 'StrengthOut',
 'DegreeIn',
 'DegreeOut',
 'Hub',
 'Authority',
 'Katz'], axis=1, inplace=True)

In [41]:
df_top15

,PageRank,Eigenvector,Degree,Strength,Multiplexity,Kcore,TEU
Label,,,,,,,
Rotterdam,1.000000,1.000000,553.0,553.0,1.00,36.0,14513
Antwerp,0.683192,0.842618,384.0,384.0,1.00,36.0,11100
Hamburg,0.508203,0.717935,297.0,297.0,1.00,36.0,8730
Bremerhaven,0.382071,0.586279,243.0,243.0,0.67,36.0,5467
Valencia,0.187045,0.729839,140.0,140.0,0.67,36.0,5104
Piraeus,0.172107,0.724053,133.0,133.0,0.33,36.0,4908
Algeciras,0.163539,0.503598,109.0,109.0,0.67,33.0,4772
Felixstowe,0.200399,0.476202,117.0,117.0,0.67,36.0,4161
Barcelona,0.147820,0.523666,110.0,110.0,0.67,36.0,3423


# Check Ranking one by one

In [42]:
df_top15_degree = df_top15.sort_values(by='Degree', ascending=False).index
df_top15_PageRank = df_top15.sort_values(by='PageRank', ascending=False).index
df_top15_eigen = df_top15.sort_values(by='Eigenvector', ascending=False).index
df_top15_strength = df_top15.sort_values(by='Strength', ascending=False).index
df_top15_multiplexity = df_top15.sort_values(by='Multiplexity', ascending=False).index
df_top15_Kcore = df_top15.sort_values(by='Kcore', ascending=False).index
df_top15_TEU = df_top15.index

# Borda Count

In [43]:
dict_degree = {df_top15_degree[x]:x+1 for x in range(len(df_top15_degree))}
dict_PageRank = {df_top15_PageRank[x]:x+1 for x in range(len(df_top15_PageRank))}
dict_eigen = {df_top15_eigen[x]:x+1 for x in range(len(df_top15_eigen))}
dict_strength = {df_top15_strength[x]:x+1 for x in range(len(df_top15_strength))}
#dict_multiplexity = {df_top15_multiplexity[x]:x+1 for x in range(len(df_top15_multiplexity))}
#dict_kcore = {df_top15_Kcore[x]:x+1 for x in range(len(df_top15_Kcore))}
dict_TEU = {df_top15_TEU[x]:x+1 for x in range(len(df_top15_TEU))}

In [44]:
# Set rank multiplexity - algorithm is different because there is same value
set_multiplexity = sorted(list(set(df_top15['Multiplexity'])), reverse=True)
dict_rank_multiplexity = {set_multiplexity[x]:x+1 for x in range(len(set_multiplexity))}
dict_multiplexity = {df_top15_multiplexity[x]:dict_rank_multiplexity[df_top15['Multiplexity'][x]] for x in range(len(df_top15_multiplexity))}

In [45]:
# Set rank kcore - algorithm is different because there is same value
set_kcore = sorted(list(set(df_top15['Kcore'])), reverse=True)
dict_rank_kcore = {set_kcore[x]:x+1 for x in range(len(set_kcore))}
dict_kcore = {df_top15_Kcore[x]:dict_rank_kcore[df_top15['Kcore'][x]] for x in range(len(df_top15_Kcore))}

In [46]:
df15_index = list(df_top15.index)

In [47]:
dict_borda = dict()
for i, j in enumerate(df15_index):
    dict_borda[j] = dict_degree[j] + dict_PageRank[j] + dict_eigen[j] + dict_multiplexity[j] + dict_kcore[j] + dict_TEU[j]

In [48]:
df_sorted_borda = pd.DataFrame(columns=['Borda_count'], index=list(dict_borda.keys()))

In [49]:
for i, j in dict_borda.items():
    df_sorted_borda.loc[i] = j

In [50]:
df_sorted_borda.sort_values(by='Borda_count')

,Borda_count
Rotterdam,6
Antwerp,10
Hamburg,16
Bremerhaven,21
Valencia,24
Piraeus,28
Felixstowe,36
Le_Havre,38
Barcelona,39
Algeciras,40


In [51]:
df_directed[df_directed['Kcore'] == 36]

,PageRank,Eigenvector,Degree,StrengthOut,DegreeIn,DegreeOut,Strength,StrengthIn,Hub,Authority,Katz,Multiplexity,Kcore
Label,,,,,,,,,,,,,
Alexandria,0.136279,0.679521,110,55,55,55,110,55,0.656177,0.672100,0.663017,0.33,36
Aliaga,0.082135,0.409648,60,30,30,30,60,30,0.395294,0.408725,0.401894,0.33,36
Antwerp,0.683192,0.842618,384,195,189,195,384,189,0.843401,0.844225,0.841935,1.00,36
Ashdod,0.132835,0.629409,106,53,53,53,106,53,0.609370,0.629770,0.623148,0.33,36
Barcelona,0.147820,0.523666,110,54,56,54,110,56,0.482136,0.515587,0.480908,0.67,36
Beirut,0.098383,0.421710,70,35,35,35,70,35,0.406061,0.413363,0.406044,0.33,36
Bremerhaven,0.382071,0.586279,243,125,118,125,243,118,0.627794,0.590359,0.622340,0.67,36
Bristol,0.106881,0.459947,81,41,40,41,81,40,0.445846,0.458366,0.454287,0.67,36
Civitavecchia,0.094981,0.395463,72,36,36,36,72,36,0.382341,0.395227,0.391968,0.33,36
